In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from matplotlib import pyplot as plt
%matplotlib inline  
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_boston
import pandas as pd
data=load_boston()

In [3]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['MEDV'] = data.target
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [6]:
print(load_boston()['DESCR'])

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

Стандартизируем наши данные, используя StandardScaler

In [7]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(df)
df_new = scaler.transform(df)

In [8]:
df_new = pd.DataFrame(df_new, index=df_new[:,0], columns = list(df))

In [9]:
df_new.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
-0.419782,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,0.159686
-0.417339,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,-0.101524
-0.417342,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,1.324247
-0.416750,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,1.182758
-0.412482,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,1.487503


Исследуем данные с помощью профайлинга.

In [8]:
import pandas_profiling

In [9]:
df_new.profile_report()

Выводы:
    пустых значений нет
    таргет-переменная распределена нормально, но имеет длинный хвост, воэтому будем исследовать ее по частям:
        меньше 2 - регрессионный анализ
        больше 2 - классификация: 0 - меньше 2, 1 - больше или равно 2

Регрессионный анализ:

In [12]:
data1 = df_new.query('MEDV < 2')

In [27]:
cols=['CRIM',
 'ZN',
 'INDUS',
 'CHAS',
 'NOX',
 'RM',
 'AGE',
 'DIS',
 'RAD',
 'TAX',
 'PTRATIO',
 'B',
 'LSTAT']

In [14]:
X=data1[cols].values

In [15]:
y=data1[['MEDV']].values

Разделим данные на тестовую и обучающую выборки

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0) #разделили на тестовые данные и обучающие (3:7)

Применим метод главных компонент (PCA)

In [29]:
from sklearn.decomposition import PCA

In [30]:
pca = PCA(n_components=4) # 4 переменные будут в результате

In [31]:
pca.fit(X_train) # обучим pca

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [32]:
print(pca.explained_variance_ratio_)  

[0.47788893 0.16763264 0.09286181 0.07907684]


In [33]:
print(pca.singular_values_)  

[17.64618437 10.4512112   7.77868007  7.17814094]


In [34]:
X_train=pca.transform(X_train) # применим pca к обучающей выборке

In [35]:
X_test=pca.transform(X_test) # применим pca к тестовой выборке

Уменьшив пространство признаков, рассмотрим следующие линейные модели:
    - линейная модель
    - линейная модель с Лассо-регуляризатором
    - линейная модель с Ридж-регуляризатором
    - линейная модель из полиномов 2ой степени
    - линейная модель из полинов 3ей степени

In [25]:
from sklearn.linear_model import LinearRegression

In [26]:
slr = LinearRegression()

slr.fit(X_train, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = slr.predict(X_train) #получаем предсказанные значения для обучающей выборки
y_test_pred = slr.predict(X_test) #получаем предсказанные значения для тестовой выборки

In [40]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score

print('MSE train linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train linear model: 0.215, test: 0.205
R^2 train linear model: 0.615, test: 0.603


In [41]:
from sklearn import linear_model

In [42]:
lml = linear_model.Lasso(alpha=0.1)

lml.fit(X_train, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = lml.predict(X_train) #получаем предсказанные значения для обучающей выборки
y_test_pred = lml.predict(X_test) #получаем предсказанные значения для тестовой выборки

In [43]:
print('MSE train Lasso linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train Lasso linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train Lasso linear model: 0.215, test: 0.205
R^2 train Lasso linear model: 0.615, test: 0.603


In [55]:
lmr = linear_model.Ridge(alpha=0.9)

lmr.fit(X_train, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = lmr.predict(X_train) #получаем предсказанные значения для обучающей выборки
y_test_pred = lmr.predict(X_test) #получаем предсказанные значения для тестовой выборки

In [56]:
print('MSE train Ridge linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train Ridge linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train Ridge linear model: 0.189, test: 0.182
R^2 train Ridge linear model: 0.661, test: 0.647


In [44]:
from sklearn.preprocessing import PolynomialFeatures

In [45]:
quadratic = PolynomialFeatures(degree=2)
cubic = PolynomialFeatures(degree=3)
X_train_quad = quadratic.fit_transform(X_train)
X_train_cubic = cubic.fit_transform(X_train)
X_test_quad = quadratic.fit_transform(X_test)
X_test_cubic = cubic.fit_transform(X_test)

In [47]:
slr = LinearRegression()

slr.fit(X_train_quad, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = slr.predict(X_train_quad) #получаем предсказанные значения для обучающей выборки
y_test_pred = slr.predict(X_test_quad) #получаем предсказанные значения для тестовой выборки

In [48]:
print('MSE train quadratic linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train quadratic linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train quadratic linear model: 0.126, test: 0.137
R^2 train quadratic linear model: 0.774, test: 0.735


In [49]:
slr = LinearRegression()

slr.fit(X_train_cubic, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = slr.predict(X_train_cubic) #получаем предсказанные значения для обучающей выборки
y_test_pred = slr.predict(X_test_cubic) #получаем предсказанные значения для тестовой выборки

In [50]:
print('MSE train cubic linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train cubic linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train cubic linear model: 0.170, test: 0.565
R^2 train cubic linear model: 0.696, test: -0.093


Лучшие результаты дали ридж регрессия и квадратичная.
Попробуем их совместить.

In [57]:
lmr = linear_model.Ridge(alpha=0.9)

lmr.fit(X_train_quad, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = lmr.predict(X_train_quad) #получаем предсказанные значения для обучающей выборки
y_test_pred = lmr.predict(X_test_quad) #получаем предсказанные значения для тестовой выборки

In [58]:
print('MSE train Ridge quadratic linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train Ridge quadratic linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train Ridge quadratic linear model: 0.126, test: 0.137
R^2 train Ridge quadratic linear model: 0.774, test: 0.735


Результаты огонь! Но попробуем подобрать гиперпараметр alpha

In [67]:
lmrcv = linear_model.RidgeCV(alphas=[0.1, 1.0, 10.0])

lmrcv.fit(X_train_quad, y_train) #обучили обучающую выборку (а что тут еще скажешь, такая вот тавтология)
y_train_pred = lmrcv.predict(X_train_quad) #получаем предсказанные значения для обучающей выборки
y_test_pred = lmrcv.predict(X_test_quad) #получаем предсказанные значения для тестовой выборки

In [68]:
print('MSE train RidgeCV quadratic linear model: {:.3f}, test: {:.3f}'.format(
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train RidgeCV quadratic linear model: {:.3f}, test: {:.3f}'.format(
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train RidgeCV quadratic linear model: 0.127, test: 0.135
R^2 train RidgeCV quadratic linear model: 0.772, test: 0.740


In [69]:
lmrcv.alpha_  

10.0

Интересно, что резльтаты для гиперпараметров, равных 0.9 и 10, не отличаются.

Приступим к задаче классификации.

In [10]:
df_new.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
-0.419782,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,0.159686
-0.417339,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,-0.101524
-0.417342,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,1.324247
-0.416750,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,1.182758
-0.412482,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,1.487503


In [11]:
def MEDV_cat(MEDV): # функция для перекодировки таргет-переменной
    if MEDV<2:
        return 0
    else:
        return 1

In [12]:
df_new['MEDV_cat']=df_new['MEDV'].apply(lambda row: MEDV_cat(row))

In [13]:
del df_new['MEDV']

In [15]:
data=df_new

У нас явный перевес в стороны категории меньше 2. Сбалансируем выборки

In [16]:
less= len(data[data['MEDV_cat'] == 0]) #посчитали, сколько наблюдений класса 0

In [17]:
more=len(data[data['MEDV_cat'] == 1]) #посчитали, сколько наблюдений класса 1

In [18]:
less, more

(475, 31)

In [19]:
less_data_indices = data[data['MEDV_cat'] == 0].index #выбрали индексы класса 0

In [20]:
more_data_indices = data[data['MEDV_cat'] == 1].index #выбрали индексы класса 1

In [21]:
random_indices = np.random.choice(less_data_indices,more, replace=False) 
#выбрали случайные индексы из класса 0 в количестве размера класса 0

In [22]:
under_sample_indices = np.concatenate([more_data_indices,random_indices]) 
#объединили индексы класса 1 и случайные индексы

In [23]:
under_sample = data.loc[under_sample_indices] # получили сбалансированную выбокру

In [24]:
less_NEW= len(under_sample[under_sample['MEDV_cat'] == 0]) #посчитали, сколько наблюдений класса 0

more_NEW=len(under_sample[under_sample['MEDV_cat'] == 1]) #посчитали, сколько наблюдений класса 1

In [25]:
less_NEW, more_NEW

(32, 31)

Разделим датасет на тестовую и обучабщие выборки.
Применим метод главных компонет.

In [36]:
X=under_sample[cols].values

y=under_sample[['MEDV_cat']].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0) #разделили на тестовые данные и обучающие (3:7)

pca = PCA(n_components=4)

pca.fit(X_train)

X_train=pca.transform(X_train)

X_test=pca.transform(X_test)

Рассмотрим три варианта обучения модели: 
 - SGDClassifier
 - RandomForestClassifier
 - DecisionTreeClassifier

In [37]:
from sklearn.linear_model import SGDClassifier
SGD = SGDClassifier()

In [38]:
SGD.fit(X_train, y_train)

/home/yellow/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [56]:
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve
y_train_pred = SGD.predict(X_train)
y_pred = SGD.predict(X_test)

print('Accuracy SGD train: {:.3f}, test: {:.3f}'.format(metrics.accuracy_score(y_train, y_train_pred),
        metrics.accuracy_score(y_test, y_pred)))
print('Recall SGD train: {:.3f}, test: {:.3f}'.format(
        metrics.recall_score(y_train, y_train_pred),
        metrics.recall_score(y_train, y_train_pred)))
print('SGD ROC AUC train: {:.3f}, test: {:.3f}'.format(
        roc_auc_score(y_train, y_train_pred),
         roc_auc_score(y_train, y_train_pred)))

Accuracy SGD train: 0.932, test: 0.842
Recall SGD train: 0.958, test: 0.958
SGD ROC AUC train: 0.929, test: 0.929


In [57]:
from sklearn.ensemble import RandomForestClassifier

In [58]:
RFC = RandomForestClassifier(random_state=0)

In [59]:
RFC.fit(X_train, y_train)  

/home/yellow/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/yellow/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [60]:
y_train_pred = RFC.predict(X_train)
y_pred = RFC.predict(X_test)

print('Accuracy RFC train: {:.3f}, test: {:.3f}'.format(metrics.accuracy_score(y_train, y_train_pred),
        metrics.accuracy_score(y_test, y_pred)))
print('Recall RFC SGD train: {:.3f}, test: {:.3f}'.format(
        metrics.recall_score(y_train, y_train_pred),
        metrics.recall_score(y_train, y_train_pred)))
print('RFC ROC AUC train: {:.3f}, test: {:.3f}'.format(
        roc_auc_score(y_train, y_train_pred),
         roc_auc_score(y_train, y_train_pred)))

Accuracy RFC train: 0.955, test: 0.842
Recall RFC SGD train: 0.917, test: 0.917
RFC ROC AUC train: 0.958, test: 0.958


In [61]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
DTC = DecisionTreeClassifier()

In [63]:
DTC.fit(X_train, y_train)  

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [64]:
y_train_pred = DTC.predict(X_train)
y_pred = DTC.predict(X_test)

print('Accuracy DTC train: {:.3f}, test: {:.3f}'.format(metrics.accuracy_score(y_train, y_train_pred),
        metrics.accuracy_score(y_test, y_pred)))
print('Recall DTC SGD train: {:.3f}, test: {:.3f}'.format(
        metrics.recall_score(y_train, y_train_pred),
        metrics.recall_score(y_train, y_train_pred)))
print('DTC ROC AUC train: {:.3f}, test: {:.3f}'.format(
        roc_auc_score(y_train, y_train_pred),
         roc_auc_score(y_train, y_train_pred)))

Accuracy DTC train: 1.000, test: 0.789
Recall DTC SGD train: 1.000, test: 1.000
DTC ROC AUC train: 1.000, test: 1.000


In [ ]:
Вывод: DecisionTreeClassifier отлично предсказывает кейсы, в которых таргет больше двух.